# Single Shot Detection

## Setup Collab

### Install nessary packages

#### torchvision install

In [ ]:
! pip install torch torchvision

#### albumentations install

In [ ]:
! pip install -U albumentations

#### opencv-python-headless install

In [ ]:
! pip install opencv-python-headless

## Libraries

### import

#### basic Imports

In [ ]:
import numpy as np

In [ ]:
import cv2


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import os

####  torch

In [ ]:
import torch

In [ ]:
from torch.utils.data import Dataset, DataLoader

#### torchvision

In [ ]:
import torchvision


In [ ]:
import torchvision.transforms as transforms

In [ ]:
from torchvision.transforms import functional as F

#### Albumentations

In [ ]:
import albumentations as A

In [ ]:
from albumentations.pytorch import ToTensorV2

#### ElementTree

In [ ]:
import xml.etree.ElementTree as ET

## Custom Functions

### Dataset Handle

#### Custom Dataset Class

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, ann_dir, transform=None):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.transform = transform
        self.imgs = list(sorted(os.listdir(img_dir)))
        self.anns = list(sorted(os.listdir(ann_dir)))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        ann_path = os.path.join(self.ann_dir, self.anns[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        boxes = []
        labels = []
        
        tree = ET.parse(ann_path)
        root = tree.getroot()
        
        for member in root.findall('object'):
            labels.append(int(member.find('name').text))
            bndbox = member.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = np.array(boxes)
        labels = np.array(labels)

        if self.transform:
            transformed = self.transform(image=img, bboxes=boxes, labels=labels)
            img = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']

        target = {}
        target['boxes'] = torch.tensor(boxes, dtype=torch.float32)
        target['labels'] = torch.tensor(labels, dtype=torch.int64)

        return img, target




#### Transform 

In [ ]:
transform = A.Compose([
    A.Resize(300, 300),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

#### collat function

In [ ]:
collate_function=lambda x: tuple(zip(*x))

#### model Evalutation

In [ ]:
def evaluate(model, val_loader, device):
    model.eval()
    val_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for images, targets in val_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            val_loss += losses.item()
            
            outputs = model(images)
            for i, output in enumerate(outputs):
                pred_boxes = output['boxes']
                pred_labels = output['labels']
                gt_boxes = targets[i]['boxes']
                gt_labels = targets[i]['labels']
                
                for box, label in zip(pred_boxes, pred_labels):
                    if label in gt_labels:
                        correct_predictions += 1
                total_predictions += len(gt_labels)
    
    val_loss /= len(val_loader)
    val_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return val_loss, val_accuracy

#### model Traing

In [ ]:
def modelTraing(model,num_epochs):
    train_losses=[]
    val_losses=[]
    val_accuracies=[]
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, targets in train_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        train_loss += losses.item()
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        train_loss /= len(train_loader)
        val_loss, val_accuracy = evaluate(model, val_loader, device)
    
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
    return {
        'val_loss':val_losses,
        'val_accuracy':val_accuracies,
        'train_losses':train_losses
    }

#### Predict

In [ ]:
def predict(model, image_path, device):
    model.eval()
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_transformed = transform(image=img)['image']
    img_transformed = img_transformed.to(device)
    with torch.no_grad():
        prediction = model([img_transformed])[0]
    
    pred_boxes = prediction['boxes'].cpu().numpy().astype(np.int32)
    pred_labels = prediction['labels'].cpu().numpy()
    
    for i in range(len(pred_boxes)):
        box = pred_boxes[i]
        label = pred_labels[i]
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(img, f'Class {label}', (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    plt.imshow(img)
    plt.show()




## Dataset

### DataSet Dir

####  Dataset dir

##### train path 

In [ ]:
trainData_set_dir={
    'image':'',
    'annotate':''
}

##### test path

In [ ]:
testData_set_dir={
    'image':'',
    'annotate':''
}

##### Validation Path

In [ ]:
valData_set_dir={
    'image':'',
    'annotate':''
}

#### Dataset

##### train

In [ ]:
train_dataset=CustomDataset(img_dir=trainData_set_dir['image'], ann_dir=trainData_set_dir['annotate'], transform=transform)

##### test

In [ ]:
test_dataset=CustomDataset(img_dir=testData_set_dir['image'], ann_dir=testData_set_dir['annotate'], transform=transform)

##### validation

In [ ]:
val_dataset=CustomDataset(img_dir=valData_set_dir['image'], ann_dir=valData_set_dir['annotate'], transform=transform)

#### Dataset Loader

##### train

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_function)

##### test

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_function)

##### val

In [ ]:
validation_loader = DataLoader(val_dataset, batch_size=4, shuffle=True, collate_fn=collate_function)

## Model

model = torchvision.models.detection.ssd300_vgg16(pretrained=True)


In [ ]:

num_classes = 32  # Replace with the number of classes in your dataset, including background
model.head.classification_head.num_classes = num_classes



In [ ]:
in_features = model.head.classification_head.cls_logits.in_channels
out_channels = model.head.classification_head.num_classes



In [ ]:
cls_logits = torch.nn.Conv2d(in_features, out_channels, kernel_size=3, padding=1)
model.head.classification_head.cls_logits = cls_logits

## Training

### Train

num_epoches=3

In [ ]:
accur_loss=modelTraing(model,num_epoches)

#### visiblize

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(num_epoches), accur_loss['train_losses'], label='Train Loss')
plt.plot(range(num_epoches), accur_loss['val_losses'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(range(num_epoches), accur_loss['val_accuracies'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')
plt.show()

## Predict